In [19]:
'''
Explore the COVID-19 data

Coronavirus Disease 2019 (COVID-19) is a disease that was first identified in Wuhan, China, 
and later spread throughout the world

The dataset is retrieved from World Health Organization (WHO) Situation Reports which includes:
date  location  new_cases  new_deaths  total_cases  total_deaths

Author: Tu Duong

Copyright © 2020 Tu Duong All Rights Reserved
'''

import os
import csv
import datetime
import urllib
import urllib.request
import urllib.error
import matplotlib.pyplot    as plt
import ipywidgets           as widgets
import pandas               as pd
import plotly.graph_objects as go
import plotly.express       as px

from plotly.subplots import make_subplots


In [24]:
# load data into pandas data frame
ourworld_url = "https://covid.ourworldindata.org/data/ecdc/full_data.csv"
df = pd.read_csv(ourworld_url)

us_covid19_data = df[ df['location']=='United States']


In [25]:
covidtracking_url='https://covidtracking.com/api/v1/us/daily.csv'

test_data = pd.read_csv(covidtracking_url)
test_data = test_data.sort_values(by='date', ascending=True)

test_data.date = [datetime.datetime.strptime(str(d), '%Y%m%d') for d in test_data.date]


In [27]:

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='Daily Tests',
                     x=test_data.date,
                     y=test_data.totalTestResultsIncrease,
                     marker_color='darkblue',
                     opacity=0.3))

fig.add_trace(go.Bar(name='US New Cases',
                     x=us_covid19_data.date,
                     y=us_covid19_data.new_cases,
                     marker_color='darkblue'))

fig.update_layout(barmode='overlay')


import math

positive_rate = us_covid19_data.new_cases / 
result = []
for d in us_covid19_data.date:
    nc = us_covid19_data[ us_covid19_data.date == d].new_cases.iloc[0]
    tt = test_data[ test_data.date == d].totalTestResultsIncrease.values
    if len(tt) == 0:
        tt = 0
    else:
        tt = tt[0]
        if math.isnan(tt):
            tt = 0

    if tt == 0:
        ratio = 0
    else:
        ratio = nc / tt

    
    result.append(ratio)
    
import plotly.express as px

fig2 = px.line(x=us_covid19_data.date, y=result, labels={'x':'Date', 'y':'Positive Rate'})
fig2.show()



fig.update_layout(title="United States", width=1000, height=500)



In [5]:
import math
#print(test_data)
result = []
for d in us_covid19_data.date:
    nc = us_covid19_data[ us_covid19_data.date == d].new_cases.iloc[0]
    tt = test_data[ test_data.date == d].totalTestResultsIncrease.values
    if len(tt) == 0:
        tt = 0
    else:
        tt = tt[0]
        if math.isnan(tt):
            tt = 0

    if tt == 0:
        ratio = 0
    else:
        ratio = nc / tt

    
    result.append(ratio)
    
import plotly.express as px

fig2 = px.line(x=us_covid19_data.date, y=result, labels={'x':'Date', 'y':'Positive Rate'})
fig2.show()